--- Day 12: Hill Climbing Algorithm ---

You try contacting the Elves using your handheld device, but the river you're following must be too low to get a decent signal.

You ask the device for a heightmap of the surrounding area (your puzzle input). The heightmap shows the local area from above broken into a grid; the elevation of each square of the grid is given by a single lowercase letter, where a is the lowest elevation, b is the next-lowest, and so on up to the highest elevation, z.

Also included on the heightmap are marks for your current position (S) and the location that should get the best signal (E). Your current position (S) has elevation a, and the location that should get the best signal (E) has elevation z.

You'd like to reach E, but to save energy, you should do it in as few steps as possible. During each step, you can move exactly one square up, down, left, or right. To avoid needing to get out your climbing gear, the elevation of the destination square can be at most one higher than the elevation of your current square; that is, if your current elevation is m, you could step to elevation n, but not to elevation o. (This also means that the elevation of the destination square can be much lower than the elevation of your current square.)

For example:

Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi

Here, you start in the top-left corner; your goal is near the middle. You could start by moving down or right, but eventually you'll need to head toward the e at the bottom. From there, you can spiral around to the goal:

v..v<<<<
>v.vv<<^
.>vv>E^^
..v>>>^^
..>>>>>^

In the above diagram, the symbols indicate whether the path exits each square moving up (^), down (v), left (<), or right (>). The location that should get the best signal is still E, and . marks unvisited squares.

This path reaches the goal in 31 steps, the fewest possible.

What is the fewest steps required to move from your current position to the location that should get the best signal?


In [1]:
def inputWrangler(filepath):
    '''
    
    '''
    
    with open(filepath, 'r') as f:
        lines = f.readlines()
    lines = [[i for i in j.strip()] for j in lines]
    grid = [[0 for i in range(len(lines[0]))] for j in range(len(lines))]
    for i, line in enumerate(lines):
        for j, letter in enumerate(line):
            if letter == 'E':
                grid[i][j] = 26
                end = (i,j)
                
            elif letter == 'S':
                grid[i][j] = 1
                start = (i,j)
                
            else:
                grid[i][j] = ord(letter)-96
    return grid, start, end

def day12a(filepath='input12.txt'):
    '''
    
    '''
    
    grid, start, end = inputWrangler(filepath)
    distances = dict()
    for i, row in enumerate(grid):
        for j , col in enumerate(row):
            distances[(i,j)] = {'path': [], 'distance': 0, 'height' : col}
            #if (i,j) == start:
            #    distances[(i,j)] = {'path': [(i,j)], 'distance': 0, 'height' : col}
    nodes = [start]
    while distances[end]['distance'] == 0:
        newNodes = []
        for node in nodes:
            x, y, height = node[0], node[1], distances[node]['height']
            
            for newNode in [(x-1, y), (x+1, y), (x, y+1), (x, y-1)]:
                if newNode in distances.keys():
                    newHeight = distances[newNode]['height']
                    if newHeight-height <= 1:
                        path = [i for i in distances[node]['path']]
                        #print(path)
                        path.append(newNode)
                        if distances[newNode]['distance'] == 0:
                            distances[newNode]['path'] = path
                            distances[newNode]['distance'] = len(path)
                            newNodes.append(newNode)
                        elif distances[newNode]['distance'] > len(path):
                            distances[newNode]['path'] = path
                            distances[newNode]['distance'] = len(path)
                            newNodes.append(newNode)
        nodes = newNodes
    #print(newNodes)
    print(distances[end]['distance'])

day12a(filepath='test12.txt')
day12a()

31
472


--- Part Two ---

As you walk up the hill, you suspect that the Elves will want to turn this into a hiking trail. The beginning isn't very scenic, though; perhaps you can find a better starting point.

To maximize exercise while hiking, the trail should start as low as possible: elevation a. The goal is still the square marked E. However, the trail should still be direct, taking the fewest steps to reach its goal. So, you'll need to find the shortest path from any square at elevation a to the square marked E.

Again consider the example from above:

Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi

Now, there are six choices for starting position (five marked a, plus the square marked S that counts as being at elevation a). If you start at the bottom-left square, you can reach the goal most quickly:

...v<<<<
...vv<<^
...v>E^^
.>v>>>^^
>^>>>>>^

This path reaches the goal in only 29 steps, the fewest possible.

What is the fewest steps required to move starting from any square with elevation a to the location that should get the best signal?



In [2]:
import numpy as np
import networkx as nx


grid, start, end = inputWrangler(filepath='input12.txt')
H = np.array(grid)

#create a 2D grid of nodes with a directed graph 
#https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.generators.classic.grid_2d_graph.html
G = nx.grid_2d_graph(*H.shape, create_using=nx.DiGraph)
print(G)

#use a list comprehension of the algorithm from a to remove illegal edges i.e delta height > 1
G.remove_edges_from([(origin,destination) for origin,destination in G.edges if H[destination] > H[origin]+1])
print(G)


#calculate all the path lengths from nodes to our end point
path = nx.shortest_path_length(G, target=end)

#check if our result from a matches
print(f'The result from a 472 = {path[start]}.')
# now calculate the minimum value for all nodes that are the lowest point      
ans = min(path[node] for node in path if H[node]==1)
print( ans)

DiGraph with 6314 nodes and 24866 edges
DiGraph with 6314 nodes and 23020 edges
The result from a 472 = 472.
465


In [145]:
#not working might come back later and find out why
def day12b(filepath='input12.txt'):
    '''
    
    '''
    import numpy as np
    grid, start, end = inputWrangler(filepath)
    distances = dict()
    for i, row in enumerate(grid):
        for j , col in enumerate(row):
            distances[(i,j)] = {'path': [], 'distance': 10000, 'height' : col}
            if (i,j) == end:
                distances[(i,j)] = {'path': [(i,j)], 'distance': -1, 'height' : col}
    startingPoints = []
    possibleDistances = dict()
    for node in distances.keys():
        if distances[node]['height'] == 1:
            startingPoints.append(node)
            possibleDistances[node] = 0
    A = np.zeros((len(grid), len(grid[0])))
    nodes = [end]
    while 0 in possibleDistances.values():
        newNodes = []
        #print(distances)
        for node in nodes:
            x, y, height = node[0], node[1], distances[node]['height']
            
            for newNode in [(x-1, y), (x+1, y), (x, y+1), (x, y-1)]:
                if newNode in distances.keys():
                    newHeight = distances[newNode]['height']
                    if height-newHeight <= 1:
                        path = [i for i in distances[node]['path']]
                        #print(path)
                        path.append(newNode)
                        if distances[newNode]['distance'] == 0:
                            distances[newNode]['path'] = path
                            distances[newNode]['distance'] = len(path)
                            newNodes.append(newNode)
                            if newNode in possibleDistances.keys():
                                possibleDistances[newNode] = len(path)
                        elif distances[newNode]['distance'] > len(path):
                            distances[newNode]['path'] = path
                            distances[newNode]['distance'] = len(path)
                            newNodes.append(newNode)
                            A[i,j] = len(path)
                            if newNode in possibleDistances.keys():
                                possibleDistances[newNode] = len(path)
        nodes = newNodes
    print(distances)
    print(possibleDistances)
    print(min(possibleDistances.values()))
    #print(distances[end]['distance'])
    #print(newNodes)
    #print(minDistance)

day12b(filepath='test12.txt')
day12b()

{(0, 0): {'path': [(2, 5), (2, 4), (1, 4), (1, 5), (1, 6), (2, 6), (3, 6), (3, 5), (3, 4), (3, 3), (2, 3), (1, 3), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 7), (2, 7), (3, 7), (4, 7), (4, 6), (4, 5), (4, 4), (4, 3), (4, 2), (3, 2), (2, 2), (1, 2), (0, 2), (0, 1), (0, 0)], 'distance': 32, 'height': 1}, (0, 1): {'path': [(2, 5), (2, 4), (1, 4), (1, 5), (1, 6), (2, 6), (3, 6), (3, 5), (3, 4), (3, 3), (2, 3), (1, 3), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 7), (2, 7), (3, 7), (4, 7), (4, 6), (4, 5), (4, 4), (4, 3), (4, 2), (3, 2), (2, 2), (1, 2), (0, 2), (0, 1)], 'distance': 31, 'height': 1}, (0, 2): {'path': [(2, 5), (2, 4), (1, 4), (1, 5), (1, 6), (2, 6), (3, 6), (3, 5), (3, 4), (3, 3), (2, 3), (1, 3), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 7), (2, 7), (3, 7), (4, 7), (4, 6), (4, 5), (4, 4), (4, 3), (4, 2), (3, 2), (2, 2), (1, 2), (0, 2)], 'distance': 30, 'height': 2}, (0, 3): {'path': [(2, 5), (2, 4), (1, 4), (1, 5), (1, 6), (2, 6), (3, 6), (3, 5), (3, 4), (3, 3), (2, 3), (1

KeyboardInterrupt: 